# Multivariate Poisson VCM

In [1]:
using GLMCopula, Random, Statistics, Test, LinearAlgebra, StatsFuns, GLM

┌ Info: Precompiling GLMCopula [c47b6ae2-b804-4668-9957-eb588c99ffbc]
└ @ Base loading.jl:1342


In [2]:
p = 3    # number of fixed effects, including intercept
m = 1    # number of variance components

# true parameter values
Random.seed!(12345)
βtrue = rand(Uniform(-2, 2), p)
Σtrue = [0.5]

d = Poisson()
link = LogLink()
D = typeof(d)
Link = typeof(link)
T = Float64

samplesize = 10000
ni = 5

gcs = Vector{GLMCopulaVCObs{T, D, Link}}(undef, samplesize)

Γ = Σtrue[1] * ones(ni, ni)

# for reproducibility I will simulate all the design matrices here
Random.seed!(12345)
X_samplesize = [randn(ni, p - 1) for i in 1:samplesize]

for i in 1:samplesize
    X = [ones(ni) X_samplesize[i]]
    η = X * βtrue
    μ = exp.(η)
    vecd = Vector{DiscreteUnivariateDistribution}(undef, ni)
    for i in 1:ni
        vecd[i] = Poisson(μ[i])
    end
    nonmixed_multivariate_dist = NonMixedMultivariateDistribution(vecd, Γ)
    # simuate single vector y
    y = Vector{Float64}(undef, ni)
    res = Vector{Float64}(undef, ni)
    rand(nonmixed_multivariate_dist, y, res)
    V = [ones(ni, ni)]
    gcs[i] = GLMCopulaVCObs(y, X, V, d, link)
end

# form VarLmmModel
gcm = GLMCopulaVCModel(gcs);

In [3]:
initialize_model!(gcm);

initializing β using Newton's Algorithm under Independence Assumption
gcm.β = [0.3403411372025437, 1.3562225658854175, -0.5008907047952436]
initializing variance components using MM-Algorithm
gcm.Σ = [0.4381828623375018]


In [4]:
gc = gcm.data[1]
β = gcm.β
Σ = gcm.Σ

1-element Vector{Float64}:
 0.4381828623375018

In [5]:
using BenchmarkTools
@benchmark update_res!($gc, $β)

BenchmarkTools.Trial: 10000 samples with 919 evaluations.
 Range (min … max):  114.703 ns … 457.548 ns  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     116.896 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   125.838 ns ±  26.647 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █▆▄▄▄     ▁▂                                                  ▁
  ██████▅█████▇▅▆▇▆▇▆▇▆▇▆█▇▅▇▆▅▆▆▅▆▅▅▅▆▅▇▅▅▆▆▅▆▅▄▅▅▄▅▃▅▅▄▃▄▅▅▅▅ █
  115 ns        Histogram: log(frequency) by time        259 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [6]:
@benchmark standardize_res!($gc)

BenchmarkTools.Trial: 10000 samples with 997 evaluations.
 Range (min … max):  129.182 ns … 413.728 ns  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     131.837 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   140.548 ns ±  28.124 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ██▆▁      ▁ ▂                                                 ▁
  ████▆▅▇▆▇▆███▆▇▇▇███▇▅▇▆▆█▇█▇▆▆▆▆▆▆▆▆▆▆▅▅▆▆▅▅▅▅▆▅▆▆▅▅▅▅▅▅▅▃▄▄ █
  129 ns        Histogram: log(frequency) by time        281 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [7]:
@benchmark std_res_differential!($gc)

BenchmarkTools.Trial: 10000 samples with 203 evaluations.
 Range (min … max):  383.759 ns …  1.665 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     395.345 ns              ┊ GC (median):    0.00%
 Time  (mean ± σ):   430.756 ns ± 95.718 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▆█▂▅    ▂▄▃         ▁                                        ▁
  ███████▇███▇▇▇▆▇▇▇▇▆█▇█▆▇▇▇▇▆▆▆▅▅▅▆▆▆▆▆▆▆▆▅▆▃▅▅▄▄▅▄▅▅▅▄▃▃▅▃▅ █
  384 ns        Histogram: log(frequency) by time       915 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [8]:
@benchmark GLMCopula.component_loglikelihood($gc)

BenchmarkTools.Trial: 10000 samples with 934 evaluations.
 Range (min … max):  106.272 ns … 475.070 ns  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     109.126 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   118.243 ns ±  26.216 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▆█▁▅   ▁▂▁▄▁         ▁                                        ▁
  █████▆▇█████▇▇▇▆▇▇▆█▇█▇▇█▇▆▆▅▅▆▆▆▆▆▆▆▅▅▆▆▆▆▆▅▅▃▅▅▅▆▅▄▅▆▅▅▂▃▄▄ █
  106 ns        Histogram: log(frequency) by time        246 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

# with just the loglikelihood

In [9]:
@benchmark loglikelihood!($gc, $β, $Σ, false, false)

BenchmarkTools.Trial: 10000 samples with 200 evaluations.
 Range (min … max):  401.680 ns …   1.622 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     412.050 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   452.796 ns ± 109.181 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █▆▃▅▂▂▂▁▄▄▁                                                   ▁
  ███████████▇█▇████▇█▇██▇▇▆▇▇▆▅▆▆▆▆▇▆▆▆▆▅▅▄▅▄▄▆▆▅▅▅▆▄▄▅▆▄▅▄▄▃▅ █
  402 ns        Histogram: log(frequency) by time        982 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

# with logl and gradient

In [10]:
@benchmark loglikelihood!($gc, $β, $Σ, true, false)

BenchmarkTools.Trial: 10000 samples with 187 evaluations.
 Range (min … max):  550.289 ns …  1.783 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     553.452 ns              ┊ GC (median):    0.00%
 Time  (mean ± σ):   592.480 ns ± 93.155 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █▄ ▅ ▃▃▂▅▂  ▁  ▂                                             ▁
  ███████████▇██▇█▇▇▆▅▅▃▆▅▅▆▅▇▇▆▃█▅▅▆▇▄▅▅▅▄▅▅▄▅▅▃▅▃▄▃▄▃▅▃▄▄▃▄▄ █
  550 ns        Histogram: log(frequency) by time      1.05 μs <

 Memory estimate: 0 bytes, allocs estimate: 0.

# with logl, gradient and hessian

In [11]:
@benchmark loglikelihood!($gc, $β, $Σ, true, true)

BenchmarkTools.Trial: 10000 samples with 10 evaluations.
 Range (min … max):  1.323 μs …  11.542 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     1.375 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   1.531 μs ± 423.915 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▆█ ▁▁▁  ▅▆▁   ▂  ▂▃▁▃                                       ▂
  ███████▅███▇▇▆██▇█████▇▄▄▅▄▄▃▃▁▁▁▃▁▄▃▃▄▅▆▄▅▄▅▄▅▅▄▃▄▄▃▄▁▅▄▅▅ █
  1.32 μs      Histogram: log(frequency) by time      3.47 μs <

 Memory estimate: 0 bytes, allocs estimate: 0.

### check each function within logl

In [12]:
@benchmark update_res!($gc, $β)

BenchmarkTools.Trial: 10000 samples with 918 evaluations.
 Range (min … max):  114.714 ns … 425.810 ns  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     119.211 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   128.381 ns ±  27.085 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▇█▆▃▆▁ ▁▁ ▁▃▃           ▁                                     ▂
  ██████▇██████▇▆▇▇▇▇██▇█▇█▆█▇▆▆▇▇▇▇▇▆▆▇▆▆▆▆▆▅▅▅▅▅▆▅▆▅▅▅▅▆▄▆▄▄▅ █
  115 ns        Histogram: log(frequency) by time        255 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [13]:
@benchmark standardize_res!($gc)

BenchmarkTools.Trial: 10000 samples with 997 evaluations.
 Range (min … max):  130.796 ns …  1.147 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     134.386 ns              ┊ GC (median):    0.00%
 Time  (mean ± σ):   145.048 ns ± 35.335 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ██▂▄ ▁▁ ▁▂ ▃      ▁                                          ▁
  ██████████▇█▇▇▇█████▇█▆████▇▇▇▇▇▇▆▆▆▆▇▆▆▆▆▆▅▅▆▅▅▅▆▄▅▅▅▄▃▄▄▄▅ █
  131 ns        Histogram: log(frequency) by time       287 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [14]:
@benchmark std_res_differential!($gc)

BenchmarkTools.Trial: 10000 samples with 203 evaluations.
 Range (min … max):  382.906 ns …  1.416 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     395.241 ns              ┊ GC (median):    0.00%
 Time  (mean ± σ):   424.356 ns ± 98.656 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▄█▁     ▁▂▁                                                  ▁
  ███▆▅▆▃▅███▆▆▅▅▅▆▅▇▅█▅█▆▇▇▇▆▄▅▅▆▅▅▅▆▆▆▆▆▆▆▆▆▅▅▅▅▅▆▄▄▄▄▅▂▄▄▄▄ █
  383 ns        Histogram: log(frequency) by time       918 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [15]:
@benchmark GLMCopula.component_loglikelihood($gc)

BenchmarkTools.Trial: 10000 samples with 934 evaluations.
 Range (min … max):  105.832 ns … 393.846 ns  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     109.100 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   118.158 ns ±  27.375 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ██▅▂▁ ▁▂▂ ▄       ▁  ▁                                        ▁
  █████▆██████▇███▇███▇█▇█▆▆▆▆▇▆▇▆▇▇▆▆▆▇▆▆▆▅▆▅▆▅▆▆▅▆▅▆▆▅▄▄▅▄▄▄▅ █
  106 ns        Histogram: log(frequency) by time        252 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [16]:
@benchmark GLMCopula.glm_gradient($gc)

BenchmarkTools.Trial: 10000 samples with 195 evaluations.
 Range (min … max):  488.605 ns …   1.983 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     502.354 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   550.037 ns ± 133.535 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▅█▄▄    ▁ ▃         ▁                                         ▁
  █████▇▇████▇▇▇▆▇▇▆██████▇▆▇▆▆▇▇▇▆▇▆▆▆▆▆▆▇▄▆▅▅▅▅▆▆▅▆▄▅▅▄▅▅▅▅▅▅ █
  489 ns        Histogram: log(frequency) by time       1.16 μs <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [17]:
@benchmark GLMCopula.glm_hessian($gc)

BenchmarkTools.Trial: 10000 samples with 818 evaluations.
 Range (min … max):  151.461 ns … 525.053 ns  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     156.205 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   169.633 ns ±  34.996 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ██▄▅▅▅▃ ▂▂▁▄▁                                                 ▂
  █████████████▇▇█▇███▇▆▇█▇▇█▇▆▇▆▇▆▇▇▆▆▆▆▆▆▇▆▅▄▆▅▆▆▆▆▅▅▅▅▅▅▅▄▂▆ █
  151 ns        Histogram: log(frequency) by time        335 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.